In [1]:
import numpy as np
import quaternion

from intrepid_environment.quadcopter_environment import QuadcopterController

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# initialize RPC Drone Controller
DT_MS = 300

quadcopter_rpc_controller = QuadcopterController(DT_MS)

Setup trajectory for drone to follow.

In [4]:
from quadcopter_control.trajectory import Trajectory, TrajectoryPoint

In [5]:
async def get_trajectory_with_initial_position():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    initial_orientation = quadcopter_rpc_controller.rotation_angles()
    initial_attitude = quaternion.from_euler_angles(*initial_orientation)
    initial_omega = quadcopter_rpc_controller.angular_velocity()
    initial_position = quadcopter_rpc_controller.position()
    initial_linear_velocity = quadcopter_rpc_controller.linear_velocity()
    initial_acceleration = quadcopter_rpc_controller.acceleration()

    starting_point = TrajectoryPoint(
        time=sim_time,
        position=initial_position,
        velocity=initial_linear_velocity,
        accel=initial_acceleration,
        attitude=initial_attitude,
        omega=initial_omega,
    )

    trajectory = Trajectory()
    trajectory.add_point(starting_point)

    return trajectory

In [6]:
def get_next_trajectory_point(trajectory, sim_time, dt, dp):
    pt = trajectory.next_trajectory_point(sim_time + dt)
    pt.position += dp
    return pt

In [7]:
trajectory_time_offset = 0
dp = np.zeros(3)

In [8]:
from quadcopter_control.controller import QuadcopterController, calculate_drone_moment_of_inertia

In [9]:
DRONE_HEIGHT = 0.028
DRONE_RADIUS = 0.05
DRONE_MASS = 0.041

In [10]:
(Ixx, Iyy, Izz) = calculate_drone_moment_of_inertia(DRONE_HEIGHT, DRONE_RADIUS, DRONE_MASS)
(Ixx, Iyy, Izz)

(2.830366666666667e-05, 2.830366666666667e-05, 5.125000000000001e-05)

## Scenario 0: Zero Gains

Test whether controller runs without controller gains set.

In [11]:
trajectory = await get_trajectory_with_initial_position()

In [12]:
quadcopter_controller = QuadcopterController(
    DT_MS / 1000.0,
    DRONE_MASS,
    Ixx, Iyy, Izz,
    max_tilt_angle=0.52,
    max_ascent_rate=3.0
)

In [13]:
trajectory = await get_trajectory_with_initial_position()

async def control_step():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    motor_commands = quadcopter_controller.run_control(
        traj_pt=trajectory.next_trajectory_point((sim_time+DT_MS)/1000.0),
        est_pos=quadcopter_rpc_controller.position(),
        est_vel=quadcopter_rpc_controller.linear_velocity(),
        est_omega=quadcopter_rpc_controller.angular_velocity(),
        est_att=quaternion.from_euler_angles(
            *quadcopter_rpc_controller.rotation_angles()
        ),
    )
    print("motor commands: ", motor_commands)
    quadcopter_rpc_controller.control_motors(*motor_commands)
    quadcopter_rpc_controller.step()

In [14]:
# await control_step()

## Scenario 1: Altitude Control

Tune gains until drone stays in place. on a desired height.

In [15]:
trajectory = await get_trajectory_with_initial_position()

In [16]:
# add 500 datapoints so drone stays in place
sim_time = await quadcopter_rpc_controller.simulation_time()

n_episodes = 1000

for i in range(0, n_episodes):
    trajectory.add_point(TrajectoryPoint(
        time=(sim_time + (i*DT_MS) / 1000.0),
        position=np.array([0.0, 0.0, 5.0]),
        velocity=np.zeros(3),
        accel=np.zeros(3),
        attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
        omega=np.array([0.0, 0.0, 0.0])
    ))

Set controller gains. Test only integrated error of altitude controller.

In [17]:
kp_att_y: 0.5
kd_att_y: 0.2
kp_att_rp: 0.5
kd_att_rp: 0.1
kp_vel_xy: 2.
kd_vel_xy: 0.5
kp_pos_xy: 1.
kd_pos_xy: 0.2
ki_pos_xy: 0.
kp_yaw: 5.
kd_yaw: 1.
kp_z: 10.
ki_z: 5.
kd_z: 5.

In [18]:
quadcopter_controller.set_gains(
    kp_pqr=np.array([95.0, 95.0, 6.0]),
    ki_pos_z=5.0,
    kappa=2.0,
)

In [19]:
async def control_step():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    traj_pt=trajectory.next_trajectory_point((sim_time+DT_MS)/1000.0)

    (t_pos, t_vel, t_acc) = traj_pt.position, traj_pt.velocity, traj_pt.accel

    traj_pt = trajectory.next_trajectory_point((sim_time+DT_MS)/1000.0)
    est_pos = quadcopter_rpc_controller.position()
    est_vel = quadcopter_rpc_controller.linear_velocity()
    est_att = quaternion.from_euler_angles(
        *quadcopter_rpc_controller.rotation_angles()
    )

    print("-------------------")
    print("altitude controller")
    print("-------------------")

    print("trajectory point: ", t_pos)
    print("actual position: ", est_pos)

    thrust = quadcopter_controller.altitude_control(
        t_pos[2],
        t_vel[2],
        est_pos[2],
        est_vel[2],
        est_att,
        t_acc[2],
        quadcopter_controller.dt
    )

    motor_commands = quadcopter_controller.generate_motor_commands(thrust, np.zeros(3))
    print("motor_commands", motor_commands)
    # intrepid-sim uses linear model for motor commands
    motor_commands *= 170.0
    quadcopter_rpc_controller.control_motors(*motor_commands)
    quadcopter_rpc_controller.step()

In [20]:
for _ in range(5000):
    await control_step()

-------------------
altitude controller
-------------------
trajectory point:  [0. 0. 5.]
actual position:  [-6.4026330e-09  1.4789308e-08 -8.1841790e-05]
pos_z_err 5.00008184179
vel_z_err -3.2363263e-05
self.integrated_altitude_error 0.5000081841790001
u1_bar 2.5000409208950005
acc -7.30995929847384
clipped_acc -7.30995929847384
motor_commands [-0.07492708 -0.07492708 -0.07492708 -0.07492708]
-------------------
altitude controller
-------------------
trajectory point:  [0. 0. 5.]
actual position:  [ 1.0104626e-08  1.0243184e-10 -1.5030609e-04]
pos_z_err 5.00015030609
vel_z_err -3.2939824e-05
self.integrated_altitude_error 1.000023214788
u1_bar 5.000116073939999
acc -4.809883948568437
clipped_acc -4.809883948568437
motor_commands [-0.04930131 -0.04930131 -0.04930131 -0.04930131]
-------------------
altitude controller
-------------------
trajectory point:  [0. 0. 5.]
actual position:  [ 2.1840714e-08  1.5968652e-08 -2.1912060e-04]
pos_z_err 5.0002191206
vel_z_err -3.3172128e-05
self.i

ReplyError: Error 100: entity not found (temporary: True)